In [0]:
## import required packages
import numpy as np
import sympy as sp
import plotly.graph_objects as go

In [0]:
# Transformation matrix that defines pirtoation about an axis in 3-space
def pirotation (m):
    m = m/np.linalg.norm(m)
    m1, m2, m3 = m.item(0), m.item(1), m.item(2)
    P = np.matrix([[2*m1**2-1, 2*m1*m2, 2*m1*m3],[2*m2*m1, 2*m2**2-1, 2*m2*m3],[2*m3*m1, 2*m3*m2, 2*m3**2-1]])
    return P
    # P is the matrix that defines pirotation about m

def hklplane(K,Pm,s,n):
  # ax + by + cz + d = 0 is the equation of a plane
  # normal to this plane is (a,b,c)
  normal = K @ np.linalg.inv(Pm)
  a,b,c = normal
  
  if a != 0:
    # x, y, z
    point = Pm @ [1/a,0,0]
  elif b != 0:
    # x, y, z
    point = Pm @ [0,1/b,0]
  elif c != 0:
    # x, y, z
    point = Pm @ [0,0,1/c]
  # d = - xa - yb - zc  
  #d = -np.sum(point*normal)

  xx,yy = np.meshgrid(np.arange(-n,n),np.arange(-n,n))
  # z = (-a*x - a * y - d)/c
  zz = (-normal[0]*xx - normal[1]*yy - s)/normal[2]
  return xx, yy, zz
  

In [0]:
# define 3 symbolic variables
beta, gamma, Gamma = sp.symbols("beta gamma Gamma")

In [0]:
## Required variables for all type II twinning mode in m-unit cell framework

# Matrix transformation from m-space to 3-space
Pm = sp.Matrix([[1.,beta*sp.cos(Gamma),0.],[0.,beta*sp.sin(Gamma),0.],[0.,0.,gamma]])
Pm = sp.lambdify((beta,gamma,Gamma),Pm,"numpy")

# Irrational component of twinnning mode for k1 = (011)
q1 = 2*beta*sp.cos(Gamma)/(gamma**2 - beta**2)
q1 = sp.lambdify((beta,gamma,Gamma),q1,"numpy")
r1 = 2*beta*gamma**2 * sp.cos(Gamma) / (gamma**2 - (beta*sp.sin(Gamma))**2)
r1 = sp.lambdify((beta,gamma,Gamma),r1,"numpy")

# Irrational component of twinnning mode for k1 = (101)
q2 = 2*beta*sp.cos(Gamma)/(gamma**2 - 1)
q2 = sp.lambdify((beta,gamma,Gamma),q2,"numpy")
r2 = 2*gamma**2*sp.cos(Gamma)/(beta*(gamma**2 - sp.sin(Gamma)**2))
r2 = sp.lambdify((beta,gamma,Gamma),r2,"numpy")

In [0]:
## Lattice parameters of monoclinic unit cell in m-space framework
a, b, c, Beta = 1, 0.85 , 0.7, np.radians(96)

## Numerical output of formal solution
Pm = Pm(b/a,c/a,Beta)
q1, r1 = q1(b/a,c/a,Beta), r1(b/a,c/a,Beta)
q2, r2 = q2(b/a,c/a,Beta), r2(b/a,c/a,Beta)

# Type II Twinning Mode

$\mathbf{K}_1 = \mathbf{k}_2 = (q_1 \bar{1} 1)_m$

$\mathbf{K}_2 = \mathbf{k}_1 = (011)_m$

$\mathbf{\eta}_1 = \mathbf{\gamma}_2 = [0\bar{1} \bar{1}]_m$

$\mathbf{\eta}_2 = \mathbf{\gamma}_1 = [\overline{r_1} 1 \bar{1}]_m$

In [0]:
# Define the rotation axis
k1, k2, eta1 = [0,1,1], [q1,-1,1], [0,-1,-1]
# rotation axis for type I
m_typeI = k1 @ np.linalg.inv(Pm)
# rotation axis for type II
m_typeII = Pm @ eta1

# no of unit cells
n = 5

In [0]:
## Define parent lattice, twin lattice and 2-fold axis
R = pirotation(m_typeI)
X, Y, Z = np.meshgrid(np.arange(-n,n),np.arange(-n,n),np.arange(-n,n))
M = np.row_stack((X.flatten(),Y.flatten(),Z.flatten()))
parent_lattice = Pm @ M
twin_lattice = np.asarray(np.linalg.inv(R) @ Pm @ M)

In [8]:
## Plot the figure
# call
fig = go.Figure()

# parent lattice
fig.add_trace(go.Scatter3d(
    x=parent_lattice[0,:],y=parent_lattice[1,:],z=parent_lattice[2,:],
    mode='markers',name='Matrix',
    marker=dict(size=11, opacity=0.85)
    ))

# twin lattice
fig.add_trace(go.Scatter3d(
    x=twin_lattice[0,:],y=twin_lattice[1,:],z=twin_lattice[2,:],
    mode='markers',name='Twin',
    marker=dict(size=10,opacity=0.5)
    ))

# sets of K1 planes
xx,yy,zz = hklplane(k1,Pm,-2,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))

xx,yy,zz = hklplane(k1,Pm,-1,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))

xx,yy,zz = hklplane(k1,Pm,0,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))

xx,yy,zz = hklplane(k1,Pm,1,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))

xx,yy,zz = hklplane(k1,Pm,2,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))

# figure aspect ratio, range, and no. of ticks
scenes = dict(
    aspectratio=dict(x=1, y=1, z=1),
    xaxis = dict(nticks=1, range=[-6,6],),
    yaxis = dict(nticks=1, range=[-6,6],),
    zaxis = dict(nticks=1, range=[-6,6],)
  )

# camera view
camera = dict(
    eye=dict(x=3,y=0,z=0),
    up = dict(x=0,y=0,z=1)
  )

# title
name = 'Type I Twinning'

# figure layout
fig.update_layout(height=800, width=800,
    scene = scenes,
    scene_camera = camera,
    title = name
  )

fig.show()

In [0]:
## Define parent lattice, twin lattice and 2-fold axis
R = pirotation(m_typeII)
X, Y, Z = np.meshgrid(np.arange(-n,n),np.arange(-n,n),np.arange(-n,n))
M = np.row_stack((X.flatten(),Y.flatten(),Z.flatten()))
parent_lattice = Pm @ M
twin_lattice = np.asarray(np.linalg.inv(R) @ Pm @ M)

In [11]:
## Plot the figure
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=parent_lattice[0,:],y=parent_lattice[1,:],z=parent_lattice[2,:],
    mode='markers',name='Matrix',
    marker=dict(size=11, opacity=0.9)
    ))
fig.add_trace(go.Scatter3d(
    x=twin_lattice[0,:],y=twin_lattice[1,:],z=twin_lattice[2,:],
    mode='markers',name='Twin',
    marker=dict(size=10,opacity=0.5)
    ))
xx,yy,zz = hklplane(k2,Pm,-2,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))
xx,yy,zz = hklplane(k2,Pm,-1,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))
xx,yy,zz = hklplane(k2,Pm,0,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))
xx,yy,zz = hklplane(k2,Pm,1,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))
xx,yy,zz = hklplane(k2,Pm,2,n)
fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),opacity=0.2,color='blue'))
fig.update_layout(scene=dict(
    aspectratio=dict(x=1, y=1, z=1),
    xaxis = dict(nticks=1, range=[-6,6],),
    yaxis = dict(nticks=1, range=[-6,6],),
    zaxis = dict(nticks=1, range=[-6,6],)
    ))
fig.show()